In [25]:
!nvidia-smi

Sat Mar 27 09:43:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [26]:
from google.colab import drive
#drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [27]:
#!unzip gdrive/MyDrive/idao_data/IDAO_2021_oski.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: IDAO_2021_oski/data/track_1/idao_dataset/train/ER/-1.4537824392318726__CYGNO_60_40_ER_10_keV_930V_30cm_IDAO_iso_crop_hist_pic_run3_ev102;1.png  
  inflating: __MACOSX/IDAO_2021_oski/data/track_1/idao_dataset/train/ER/._-1.4537824392318726__CYGNO_60_40_ER_10_keV_930V_30cm_IDAO_iso_crop_hist_pic_run3_ev102;1.png  
  inflating: IDAO_2021_oski/data/track_1/idao_dataset/train/ER/-1.0829859972000122__CYGNO_60_40_ER_30_keV_930V_30cm_IDAO_iso_crop_hist_pic_run3_ev201;1.png  
  inflating: __MACOSX/IDAO_2021_oski/data/track_1/idao_dataset/train/ER/._-1.0829859972000122__CYGNO_60_40_ER_30_keV_930V_30cm_IDAO_iso_crop_hist_pic_run3_ev201;1.png  
  inflating: IDAO_2021_oski/data/track_1/idao_dataset/train/ER/1.200452208518982__CYGNO_60_40_ER_30_keV_930V_30cm_IDAO_iso_crop_hist_pic_run3_ev140;1.png  
  inflating: __MACOSX/IDAO_2021_oski/data/track_1/idao_dataset/train/ER/._1.200452208518982__CYGNO_60_40_ER_30_keV_930V_30

In [28]:
!pip install transformers
!pip install timm
!pip install albumentations==0.4.6


     |████████████████████████████████| 2.0MB 6.7MB/s 
     |████████████████████████████████| 890kB 26.8MB/s 
     |████████████████████████████████| 3.2MB 57.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=0b4eb6fc0a7041ce7889e245007745cff7ca00a8894bb5631d2b635e869476ee
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 296kB 7.1MB/s 
     |████████████████████████████████| 122kB 8.9MB/s 
     |████████████████████████████████| 952kB 13.1MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-cp37-none-any.whl size=65163 sha256=5bd953de3f188c2fc65a8c04638bd7f2722e28b219cbe20800e6dcb40191de3e
  Stored in directory: /root/.cache/pip/wheels/c7/f4/89/56d1bee5c421c36c1a951eeb4adcc32fbb82f5344c086efa14
Successfully built albumentations
  Found existing installation: imgaug 0.2.9
    Uninstalling 

In [30]:
import sys
sys.path.append("IDAO_2021_oski/src")

from collections import defaultdict
import os
import random

import numpy as np
import pandas as pd
import yaml
import shutil
from sklearn.model_selection import train_test_split, StratifiedKFold

from torch.utils.data import DataLoader
from torch import nn
import torch

from transformers import get_linear_schedule_with_warmup

from datasets import SimpleDataset
from models import Wrapper, MixUp
from pipeline_utils import training, pseudolabeling
from models import ENCODER_PARAMS



In [32]:
PATH_TO_CFG = "IDAO_2021_oski/config/config.yaml"
with open(PATH_TO_CFG, "r") as file:
    config = yaml.load(file)

DATA_ROOT = config["general"]["data_root"]

def fix_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

fix_seed(config["general"]["seed"])
device_ids = [str(id) for id in config["general"]["device_ids"]]
ids = ",".join(device_ids)
DEVICE = torch.device(f"cuda:{ids}")

In [36]:
DATA_ROOT = "IDAO_2021_oski/data/track_1/idao_dataset"

In [37]:
train = pd.read_csv(os.path.join(DATA_ROOT, "train.csv"), index_col=0)

val_private = pd.read_csv(os.path.join(DATA_ROOT, "val_private.csv"), index_col=0)

In [38]:
train["file_path"] = train["file_path"].str.replace("/media/paniquex/samsung_2tb/IDAO_2021_oski", "IDAO_2021_oski")
val_private["file_path"] = val_private["file_path"].str.replace("/media/paniquex/samsung_2tb/IDAO_2021_oski", "IDAO_2021_oski")

In [39]:
train.sample(10)

,0,1,file_path,target
4030,NR,20,IDAO_2021_oski/data/track_1/idao_dataset/train...,8
12593,ER,30,IDAO_2021_oski/data/track_1/idao_dataset/train...,4
7515,ER,3,IDAO_2021_oski/data/track_1/idao_dataset/train...,3
1608,NR,6,IDAO_2021_oski/data/track_1/idao_dataset/train...,11
9627,ER,30,IDAO_2021_oski/data/track_1/idao_dataset/train...,4
7460,ER,30,IDAO_2021_oski/data/track_1/idao_dataset/train...,4
8568,ER,10,IDAO_2021_oski/data/track_1/idao_dataset/train...,1
7526,ER,10,IDAO_2021_oski/data/track_1/idao_dataset/train...,1
35,NR,20,IDAO_2021_oski/data/track_1/idao_dataset/train...,8
1749,NR,1,IDAO_2021_oski/data/track_1/idao_dataset/train...,6


In [40]:
test = pd.read_csv(os.path.join(DATA_ROOT, "test.csv"), index_col=0)
test["file_path"] = test["file_path"].str.replace("/media/paniquex/samsung_2tb/IDAO_2021_oski", "IDAO_2021_oski")
test.sample(5)

,file_path,type
7498,IDAO_2021_oski/data/track_1/idao_dataset/priva...,private
4411,IDAO_2021_oski/data/track_1/idao_dataset/priva...,private
2246,IDAO_2021_oski/data/track_1/idao_dataset/priva...,private
16149,IDAO_2021_oski/data/track_1/idao_dataset/priva...,private
10109,IDAO_2021_oski/data/track_1/idao_dataset/priva...,private


In [41]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder() 



mask_NR = (train["0"] == "NR") & ((train["1"] == 1) | (train["1"] == 6) | (train["1"] == 20))
mask_ER = (train["0"] == "ER") & ((train["1"] == 3) | (train["1"] == 10) | (train["1"] == 30))
train = train[mask_NR | mask_ER]
train.index = pd.RangeIndex(0, len(train.index))
if config["general"]["task_type"] == "regression":
    train["target"] = train["1"]
    val_private["target"] = val_private["1"]
elif config["general"]["task_type"] == "classification":
    train["target"] = le.fit_transform(train["target"])
#     val_private["target"] = le.fit_transform(val_private)
elif config["general"]["task_type"] == "joint":
    train["target_regression"] = train["1"]
    train["target_classification"] = le.fit_transform(train["0"])
    train["target"] = train["target_regression"].astype(str) + "_" + train["target_classification"].astype(str)
    
    val_private["target_regression"] = val_private["1"]
    val_private["target_classification"] = le.fit_transform(val_private["0"])


kfold = StratifiedKFold(n_splits=config["training"]["n_folds"], shuffle=True,
                        random_state=config["general"]["seed"])
for fold, (t_idx, v_idx) in enumerate(kfold.split(train, train["target"])):
    train.loc[v_idx, "kfold"] = fold


    
train.to_csv(os.path.join(DATA_ROOT, "train", "train_folds.csv"))



In [42]:
val_private

,0,1,file_path,target,target_regression,target_classification
2719,NR,30,IDAO_2021_oski/data/track_1/idao_dataset/train...,10,30,1
3783,NR,3,IDAO_2021_oski/data/track_1/idao_dataset/train...,9,3,1
4874,NR,10,IDAO_2021_oski/data/track_1/idao_dataset/train...,7,10,1
4915,NR,30,IDAO_2021_oski/data/track_1/idao_dataset/train...,10,30,1
5485,NR,3,IDAO_2021_oski/data/track_1/idao_dataset/train...,9,3,1
5781,NR,10,IDAO_2021_oski/data/track_1/idao_dataset/train...,7,10,1
7132,ER,20,IDAO_2021_oski/data/track_1/idao_dataset/train...,2,20,0
7397,ER,20,IDAO_2021_oski/data/track_1/idao_dataset/train...,2,20,0
8012,ER,1,IDAO_2021_oski/data/track_1/idao_dataset/train...,0,1,0
8142,ER,6,IDAO_2021_oski/data/track_1/idao_dataset/train...,5,6,0


In [43]:
!pip install --upgrade --force-reinstall --no-deps albumentations

     |████████████████████████████████| 81kB 5.7MB/s 
  Found existing installation: albumentations 0.4.6
    Uninstalling albumentations-0.4.6:
      Successfully uninstalled albumentations-0.4.6


In [44]:
import albumentations
from albumentations import *
from albumentations.pytorch import ToTensorV2


transforms_train = albumentations.Compose([
    ColorJitter (brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, always_apply=False, p=0.5),
    ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    CenterCrop(height=400,
               width=400),
    Resize(*config["preprocessing"]["img_size"]),
    Normalize(
         mean=[0.485, 0.456, 0.406],
         std=[0.229, 0.224, 0.225],
     ),
    ToTensorV2()
])

transforms_val = albumentations.Compose([
    CenterCrop(height=400,
               width=400),
    Resize(*config["preprocessing"]["img_size"]),
    Normalize(
         mean=[0.485, 0.456, 0.406],
         std=[0.229, 0.224, 0.225],
     ),
    ToTensorV2()
])

In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F
    
    
def focal_loss(input, target, focus=2.0, raw=False):

    if raw:
        input = torch.sigmoid(input)

    eps = 1e-7

    prob_true = input * target + (1 - input) * (1 - target)
    prob_true = torch.clamp(prob_true, eps, 1-eps)
    modulating_factor = (1.0 - prob_true).pow(focus)

    return (-modulating_factor * prob_true.log()).mean()


from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss


class AngularPenaltySMLoss(nn.Module):

    def __init__(self, in_features, out_features, loss_type="cosface", eps=1e-7, s=None, m=None):
        '''
        Angular Penalty Softmax Loss
        Three 'loss_types' available: ['arcface', 'sphereface', 'cosface']
        These losses are described in the following papers: 
        
        ArcFace: https://arxiv.org/abs/1801.07698
        SphereFace: https://arxiv.org/abs/1704.08063
        CosFace/Ad Margin: https://arxiv.org/abs/1801.05599
        '''
        super(AngularPenaltySMLoss, self).__init__()
        loss_type = loss_type.lower()
        assert loss_type in  ['arcface', 'sphereface', 'cosface']
        if loss_type == 'arcface':
            self.s = 64.0 if not s else s
            self.m = 0.5 if not m else m
        if loss_type == 'sphereface':
            self.s = 64.0 if not s else s
            self.m = 1.35 if not m else m
        if loss_type == 'cosface':
            self.s = 30.0 if not s else s
            self.m = 0.4 if not m else m
        self.loss_type = loss_type
        self.in_features = in_features
        self.out_features = out_features
        self.fc = nn.Linear(in_features, out_features, bias=False)
        self.eps = eps

    def forward(self, x, labels):
        '''
        input shape (N, in_features)
        '''
        assert len(x) == len(labels)
        assert torch.min(labels) >= 0
        assert torch.max(labels) < self.out_features
        
        for W in self.fc.parameters():
            W = F.normalize(W, p=2, dim=1)

        x = F.normalize(x, p=2, dim=1)
#         print(x.shape)
        wf = self.fc(x)
        if self.loss_type == 'cosface':
            numerator = self.s * (torch.diagonal(wf.transpose(0, 1)[labels]) - self.m)
        if self.loss_type == 'arcface':
            numerator = self.s * torch.cos(torch.acos(torch.clamp(torch.diagonal(wf.transpose(0, 1)[labels]), -1.+self.eps, 1-self.eps)) + self.m)
        if self.loss_type == 'sphereface':
            numerator = self.s * torch.cos(self.m * torch.acos(torch.clamp(torch.diagonal(wf.transpose(0, 1)[labels]), -1.+self.eps, 1-self.eps)))
        excl = torch.cat([torch.cat((wf[i, :y], wf[i, y+1:])).unsqueeze(0) for i, y in enumerate(labels)], dim=0)
        denominator = torch.exp(numerator) + torch.sum(torch.exp(self.s * excl), dim=1)
        L = numerator - torch.log(denominator)
        return -torch.mean(L), wf

In [55]:
config["training"]["loss"] = {"reg": "L1", "clf": "BCE"}

In [56]:
EPOCHS = config["training"]["n_epochs"]


criterion_aam = None
if config["general"]["task_type"] == "classification":
    if config["training"]["loss"]["clf"] == "FOCAL":
        criterion = focal_loss
    elif config["training"]["loss"]["clf"] == "AAM":
        criterion = "AAM"
        criterion_aam = AngularPenaltySMLoss
    elif config["training"]["loss"]["clf"] == "BCE":
        criterion = nn.BCELoss()
elif config["general"]["task_type"] == "regression":
    if config["training"]["loss"]["reg"] == "L1":
        criterion = nn.L1Loss()
    elif config["training"]["loss"]["reg"] == "L2":
        criterion = nn.MSELoss()
elif config["training"]["loss"] == "L1":
    criterion = nn.L1Loss()
elif config["general"]["task_type"] == "joint":
    criterion = {}
    if config["training"]["loss"]["clf"] == "FOCAL":
        criterion["clf"] = focal_loss
    elif config["training"]["loss"]["clf"] == "AAM":
        criterion["clf"] = "AAM"
        criterion_aam = AngularPenaltySMLoss
    elif config["training"]["loss"]["clf"] == "BCE":
        criterion["clf"] = nn.BCELoss()
    if config["training"]["loss"]["reg"] == "L1":
        criterion["reg"] = nn.L1Loss()
    elif config["training"]["loss"]["reg"] == "L2":
        criterion["reg"] = nn.MSELoss()

In [57]:
model_names = None
if config["training"]["finetune"]:
    model_names = [name for name in os.listdir(config['training']['models_dir']) if name.find("best_model_fold") != -1]
    model_names = sorted(model_names)

In [58]:
criterion

{'clf': BCELoss(), 'reg': L1Loss()}

In [62]:
config["general"]["out_path"] = "IDAO_2021_oski/experiments/resnest14d_1e-4_joint_BCE_L1/"
config["preprocessing"]["img_size"] = [224, 224]
config["training"]["dataloader"]["batch_size"] = 64
config["training"]["n_folds"] = 3

In [ ]:
try:
    shutil.rmtree(config["general"]["out_path"])
except:
    pass

try:
    os.mkdir(config["general"]["out_path"])
except:
    pass


if config["general"]["task_type"] == "regression":
    config["general"]["classes_num"] = 1
elif config["general"]["task_type"] == "joint":
    config["general"]["classes_num"] = 2
    
    
samples2preds_all = {}
samples2trues_all = {}


for i in range(config["training"]["n_folds"]):
    model_name = config["general"]["model_name"]
    model = None
    model = ENCODER_PARAMS[model_name]["init_op"]()
    model = Wrapper(model, feat_module=None, classes_num=config["general"]["classes_num"],
                    model_name=model_name,
                    spec_augmenter=None, 
                    mixup_module=None,
                    task_type=config["general"]["task_type"],
                    activation_func=config["training"]["activation_func"],
                    criterion_aam=criterion_aam)
    model.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=config["training"]["lr"])
#         optimizer = Ranger(model.parameters(),
#                lr=config["training"]["lr"],
#                betas=(.90, 0.999), k=4)
    train_dataset = SimpleDataset(df=train[train["kfold"] != fold], mode="train",
                                  transform=transforms_train, classes_num=config["general"]["classes_num"],
                                  task_type=config["general"]["task_type"])

    val_dataset = SimpleDataset(df=train[train["kfold"] == fold], mode="val",
                                transform=transforms_val, classes_num=config["general"]["classes_num"],
                                task_type=config["general"]["task_type"])
    val_private_dataset = SimpleDataset(df=val_private, mode="val",
                                        transform=transforms_val, classes_num=config["general"]["classes_num"],
                                        task_type=config["general"]["task_type"])
    
    train_dataloader = DataLoader(train_dataset,
                                  **config["training"]["dataloader"])
    val_dataloader = DataLoader(val_dataset,
                                **config["validation"]["dataloader"])
    val_private_dataloader = DataLoader(val_private_dataset,
                                        **config["validation"]["dataloader"])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                      T_max=(config["training"]["n_epochs"] - config["training"]["n_epochs_flat"])  * len(train_dataloader),
                                                      eta_min=1e-8)    
    samples2preds, samples2trues = training(EPOCHS=EPOCHS, model=model,
                                            train_dataloader=train_dataloader, 
                                            val_dataloaders_dct={"val_dataloader": val_dataloader,
                                                                  "val_private_dataloader": val_private_dataloader},
                                            DEVICE=DEVICE, criterion=criterion,
                                            optimizer=optimizer, scheduler=scheduler,
                                            config=config, fold=i,
                                            task_type=config["general"]["task_type"], CONFIG_PATH=PATH_TO_CFG)
    samples2preds_all.update(samples2preds)
    samples2trues_all.update(samples2trues)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0%|          | 0/112 [00:00<?, ?it/s]

Predict test without augmentations


100%|██████████| 112/112 [00:13<00:00,  8.13it/s]


EARLY STOPPING COUNTER: 1/10


  0%|          | 0/1 [00:00<?, ?it/s]

...

100%|██████████| 112/112 [00:12<00:00,  9.02it/s]


EARLY STOPPING COUNTER: 1/10


  0%|          | 0/1 [00:00<?, ?it/s]

Predict test without augmentations


  0%|          | 0/167 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
0.1715:  92%|█████████▏| 154/167 [01:44<00:08,  1.55it/s]

In [68]:


if config["general"]["task_type"] == "regression":
    config["general"]["classes_num"] = 1
elif config["general"]["task_type"] == "joint":
    config["general"]["classes_num"] = 2
    
    
samples2preds_all = {}
samples2trues_all = {}
LR = config["training"]["lr"]
flag_LR = True

for j in range(config["pseudo"]["n_epochs"]):
    models = []
    for i in range(config["training"]["n_folds"]):
        model_name = config["general"]["model_name"]
        model = None
        model = ENCODER_PARAMS[model_name]["init_op"]()
        model = Wrapper(model, feat_module=None, classes_num=config["general"]["classes_num"],
                        model_name=model_name,
                        spec_augmenter=None, 
                        mixup_module=None,
                        task_type=config["general"]["task_type"],
                        activation_func=config["training"]["activation_func"],
                        criterion_aam=criterion_aam)
        model.to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    #         optimizer = Ranger(model.parameters(),
    #                lr=config["training"]["lr"],
    #                betas=(.90, 0.999), k=4)
        train_dataset = SimpleDataset(df=train[train["kfold"] != fold], mode="train",
                                      transform=transforms_train, classes_num=config["general"]["classes_num"],
                                      task_type=config["general"]["task_type"])

        val_dataset = SimpleDataset(df=train[train["kfold"] == fold], mode="val",
                                    transform=transforms_val, classes_num=config["general"]["classes_num"],
                                    task_type=config["general"]["task_type"])
        val_private_dataset = SimpleDataset(df=val_private, mode="val",
                                            transform=transforms_val, classes_num=config["general"]["classes_num"],
                                            task_type=config["general"]["task_type"])
        
        train_dataloader = DataLoader(train_dataset,
                                      **config["training"]["dataloader"])
        val_dataloader = DataLoader(val_dataset,
                                    **config["validation"]["dataloader"])
        val_private_dataloader = DataLoader(val_private_dataset,
                                            **config["validation"]["dataloader"])
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                          T_max=(config["training"]["n_epochs"] - config["training"]["n_epochs_flat"])  * len(train_dataloader),
                                                          eta_min=1e-8)    
        samples2preds, samples2trues, model = training(EPOCHS=EPOCHS, model=model,
                                                train_dataloader=train_dataloader, 
                                                val_dataloaders_dct={"val_dataloader": val_dataloader,
                                                                      "val_private_dataloader": val_private_dataloader},
                                                DEVICE=DEVICE, criterion=criterion,
                                                optimizer=optimizer, scheduler=scheduler,
                                                config=config, fold=i, pseudo_iter=j,
                                                task_type=config["general"]["task_type"], CONFIG_PATH=PATH_TO_CFG)
        models.append(model)
        samples2preds_all.update(samples2preds)
        samples2trues_all.update(samples2trues)

    with torch.no_grad():
        train, test = pseudolabeling(models, train, test, config)
        private = (train["type"].values == "private").sum()
        public = (train["type"].values == "public").sum()
        print("Pseudo labeling epoch", j)
        print("Private ratio", private / (public + private)) 
        print("Public ratio", public / (public + private))
        if flag_LR:
            LR *= config["pseudo"]["lr_coef"]
            flag_LR = False

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



  0%|          | 0/139 [00:00<?, ?it/s]


13.5299:   0%|          | 0/139 [00:04<?, ?it/s]


13.5299:   1%|          | 1/139 [00:04<09:29,  4.13s/it]


12.3464:   1%|          | 1/139 [00:04<09:29,  4.13s/it]


12.3464:   1%|▏         | 2/139 [00:04<06:51,  3.01s/it]


12.0564:   1%|▏         | 2/139 [00:04<06:51,  3.01s/it]


12.0564:   2%|▏         | 3/139 [00:04<05:05,  2.24s/it]


11.5075:   2%|▏         | 3/139 [00:05<05:05,  2.24s/it]


11.5075:   3%|▎         | 4/139 [00:05<03:49,  1.70s/it]


10.9038:   3%|▎         | 4/139 [00

Predict test without augmentations





  1%|          | 1/186 [00:00<01:24,  2.20it/s]


  2%|▏         | 3/186 [00:00<01:01,  2.96it/s]


  3%|▎         | 5/186 [00:00<00:47,  3.83it/s]


  4%|▍         | 7/186 [00:00<00:36,  4.94it/s]


  5%|▍         | 9/186 [00:01<00:28,  6.18it/s]


  6%|▌         | 11/186 [00:01<00:24,  7.16it/s]


  7%|▋         | 13/186 [00:01<00:20,  8.62it/s]


  8%|▊         | 15/186 [00:01<00:17,  9.59it/s]


  9%|▉         | 17/186 [00:01<00:15, 10.93it/s]


 10%|█         | 19/186 [00:01<00:16, 10.30it/s]


 11%|█▏        | 21/186 [00:01<00:14, 11.60it/s]


 12%|█▏        | 23/186 [00:02<00:13, 11.91it/s]


 13%|█▎        | 25/186 [00:02<00:12, 13.16it/s]


 15%|█▍        | 27/186 [00:02<00:15, 10.26it/s]


 16%|█▌        | 30/186 [00:02<00:13, 11.80it/s]


 17%|█▋        | 32/186 [00:02<00:11, 12.91it/s]


 18%|█▊        | 34/186 [00:02<00:11, 13.49it/s]


 19%|█▉        | 36/186 [00:03<00:12, 11.87it/s]


 20%|██        | 38/186 [00:03<00:11, 12.78it/s]


 22%|██▏       | 40/186 [00:03<00

EARLY STOPPING COUNTER: 1/5
Predict test without augmentations





100%|██████████| 1/1 [00:00<00:00,  3.89it/s]



  0%|          | 0/139 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



1.6123:   0%|          | 0/139 [00:03<?, ?it/s]


1.6123:   1%|          | 1/139 [00:03<08:09,  3.55s/it]


1.4200:   1%|          | 1/139 [00:03<08:09,  3.55s/it]


1.4200:   1%|▏         | 2/139 [00:03<05:56,  2.60s/it]


1.2592:   1%|▏         | 2/139 [00:04<05:56,  2.60s/it]


1.2592:   2%|▏         | 3/139 [00:04<04:27,  1.97s/it]


1.1801:   2%|▏         | 3/139 [00:04<04:27,  1.97s/it]


1.1801:   3%|▎         | 4/139 [00:04<03:26,  1.53s/

...

EARLY STOPPING COUNTER: 1/5
Predict test without augmentations





100%|██████████| 1/1 [00:00<00:00,  3.55it/s]



  0%|          | 0/139 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



0.4497:   0%|          | 0/139 [00:04<?, ?it/s]


0.4497:   1%|          | 1/139 [00:04<09:38,  4.19s/it]


0.4278:   1%|          | 1/139 [00:04<09:38,  4.19s/it]


0.4278:   1%|▏         | 2/139 [00:04<06:57,  3.04s/it]


0.4081:   1%|▏         | 2/139 [00:05<06:57,  3.04s/it]


0.4081:   2%|▏         | 3/139 [00:05<05:10,  2.29s/it]


0.4264:   2%|▏         | 3/139 [00:05<05:10,  2.29s/it]


0.4264:   3%|▎         | 4/139 [00:05<03:53,  1.73s/

Predict test without augmentations





  1%|          | 1/186 [00:00<01:21,  2.27it/s]


  2%|▏         | 3/186 [00:00<00:59,  3.06it/s]


  3%|▎         | 5/186 [00:00<00:46,  3.86it/s]


  4%|▍         | 7/186 [00:00<00:36,  4.96it/s]


  5%|▍         | 9/186 [00:01<00:28,  6.20it/s]


  6%|▌         | 11/186 [00:01<00:22,  7.64it/s]


  7%|▋         | 13/186 [00:01<00:19,  8.97it/s]


  8%|▊         | 15/186 [00:01<00:18,  9.42it/s]


  9%|▉         | 17/186 [00:01<00:16, 10.12it/s]


 10%|█         | 19/186 [00:01<00:15, 10.56it/s]


 11%|█▏        | 21/186 [00:01<00:15, 10.63it/s]


 12%|█▏        | 23/186 [00:02<00:15, 10.75it/s]


 13%|█▎        | 25/186 [00:02<00:14, 10.97it/s]


 15%|█▍        | 27/186 [00:02<00:15, 10.46it/s]


 16%|█▌        | 29/186 [00:02<00:13, 11.36it/s]


 17%|█▋        | 31/186 [00:02<00:14, 10.88it/s]


 18%|█▊        | 33/186 [00:03<00:13, 11.45it/s]


 19%|█▉        | 35/186 [00:03<00:12, 12.23it/s]


 20%|█▉        | 37/186 [00:03<00:12, 11.67it/s]


 21%|██        | 39/186 [00:03<00

KeyboardInterrupt: ignored